<a href="https://colab.research.google.com/github/Martlgap/x-face-verification/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prerequisites

In [ ]:
# Prerequisites
!git clone https://github.com/Martlgap/x-face-verification.git
!pip install -r x-face-verification/requirements.txt
!wget https://github.com/Martlgap/octuplet-loss/releases/download/modelweights/ArcFaceOctupletLoss.tf.zip -P x-face-verification/demo
!unzip x-face-verification/demo/ArcFaceOctupletLoss.tf.zip -d x-face-verification/demo

In [2]:
# Add the Repo to PYTHONPATH
import os
os.environ['PYTHONPATH'] += ":/content/x-face-verification"

In [ ]:
# Set the Repo as working directory
%cd x-face-verification/

# Example for Confidence Score Calculation

Imports

In [4]:
import pickle
from src import ConfidenceScoreGenerator, calculate_score
from sklearn.metrics.pairwise import paired_cosine_distances

Load example embedddings and labels (Generated for XQLFW database utilizing a FaceTransformer model fine-tuned with OctupletLoss)

In [5]:
with open("./demo/embeddings1.pkl", "rb") as f:
    embeddings1 = pickle.load(f)
with open("./demo/embeddings2.pkl", "rb") as f:
    embeddings2 = pickle.load(f)
with open("./demo/labels.pkl", "rb") as f:
    labels = pickle.load(f)

Calculate Confidence Score

In [ ]:
# Calculate pairwise cosine distances
distances = paired_cosine_distances(embeddings1, embeddings2)

# Define a threshold for binary decision
THRESHOLD = 0.2250

# Initiate the confidence calculation with the distances, labels and threshold of a dataset
ConfidenceScoreGenerator = ConfidenceScoreGenerator()
sigmoid_parameters = ConfidenceScoreGenerator(cosine_distances=distances, labels=labels, threshold=THRESHOLD)

# Calculate scores for specific pairs
PAIR_ID = 0
confidence_score_raw = calculate_score(sigmoid_parameters, distances[PAIR_ID])
confidence_score = (confidence_score_raw if distances[PAIR_ID] < THRESHOLD else 1 - confidence_score_raw) * 100

# Display result
print(
    f'The prediction: "{distances[PAIR_ID] < THRESHOLD}" for pair id: {PAIR_ID} '
    f"has a confidence score of: {confidence_score:.02f}%."
    )

# Example for Generation Explanation Maps

Imports

In [17]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from src import MapGenerator, colorblend
from demo import ArcFaceOctupletLoss

Load example pair of images

In [ ]:
image_pair = (
    cv2.cvtColor(cv2.imread("./demo/img1.png"), cv2.COLOR_BGR2RGB).astype(np.float32) / 255,
    cv2.cvtColor(cv2.imread("./demo/img2.png"), cv2.COLOR_BGR2RGB).astype(np.float32) / 255,
)

# Show example image pair
fig, ax = plt.subplots(1, 2)
fig.suptitle("Example Image Pair")
ax[0].imshow(image_pair[0]), ax[1].imshow(image_pair[1])
fig.show()

Generate Explanation Maps

In [ ]:
# Instantiate the MapGenerator
MapGenerator = MapGenerator(inference_fn=ArcFaceOctupletLoss(batch_size=64))

# Generate and visualize the explanation maps
fig, ax = plt.subplots(3, 2)
fig.suptitle("Explanation Maps for Method 1, 2 and 3")
print("Method 1:")
map1_m1, map2_m1 = MapGenerator(*image_pair, method="1")  # using method 1 for explanation maps
ax[0, 0].imshow(map1_m1), ax[0, 1].imshow(map2_m1)
print("Method 2:")
map1_m2, map2_m2 = MapGenerator(*image_pair, method="2")  # using method 2 for explanation maps
ax[1, 0].imshow(map1_m2), ax[1, 1].imshow(map2_m2)
print("Method 3:")
map1_m3, map2_m3 = MapGenerator(*image_pair, method="3")  # using method 3 for explanation maps
ax[2, 0].imshow(map1_m3), ax[2, 1].imshow(map2_m3)
plt.show()

Blend the explanations maps with the original images and visualize

In [ ]:
fig, ax = plt.subplots(3, 2)
fig.suptitle("Blended Explanation Maps for Method 1, 2 and 3")
ax[0, 0].imshow(colorblend(image_pair[0], map1_m1)), ax[0, 1].imshow(colorblend(image_pair[1], map2_m1))
ax[1, 0].imshow(colorblend(image_pair[0], map1_m2)), ax[1, 1].imshow(colorblend(image_pair[1], map2_m2))
ax[2, 0].imshow(colorblend(image_pair[0], map1_m3)), ax[2, 1].imshow(colorblend(image_pair[1], map2_m3))
plt.show()